In [53]:
# !pip install mysql-connector-python

In [54]:
## Reading From init file
from mysql_connect import MySqlConnection

In [55]:
import pandas as pd
import mysql.connector
import getpass,os

## Common Functions

In [56]:
def dir_update(txt):
    txt = txt.replace("\\","/")
    return txt

In [57]:
dir_update(os.getcwd())

'd:/ML_NOTE/SQL/Python_4_sql'

## MySqlConnection Class

In [72]:
class MySqlConnection:
    def __init__ (self,host,user,database=None):
        self.host = host
        self.user = user
        self.password =  self.password = getpass.getpass(prompt="Enter your password: ")
        self.database = database
        self.conn = None
        

    
    def connect(self):
        """Establish Mysql Connection.."""
        try:
            self.conn = mysql.connector.connect(host = self.host,
                                                user = self.user,
                                                password = self.password)
            if self.conn.is_connected():
                print(f"MySql Server Connection Established...\n\nConnection Details:")

                cursor = self.conn.cursor()
                cursor.execute("select user();")
                print(f"User --> {cursor.fetchone()[0]}")

                cursor.execute("SELECT DATE_FORMAT(NOW(), '%Y-%m-%d %r') AS CurrentDateTime;")
                print(f"Date Time --> {cursor.fetchone()[0]}")
    
        except Exception as e:
            print(f"MySql Connect failed .....{e}")
            self.conn = None




    def select_database(self, db_name): 
        """Select a specific database.""" 
        if self.conn is None or not self.conn.is_connected(): 
            print("Reconnecting to MySQL...") 
            self.connect() 
        try: 
            self.conn.database = db_name 
            print(f"Connected to database: {db_name}") 
        except Exception as e:
            print(f"Failed to select database {db_name}: {e}")




    def create_datebase(self,db_name):
        """Create New Database"""
        if self.conn is None:
            print("Please Connect to the MySql Server")
            return 
        try:
            cursor = self.conn.cursor()
            cursor.execute(f"DROP DATABASE IF EXISTS {db_name}") 
            cursor.execute(f"CREATE DATABASE {db_name}")
            print(f"Database {db_name} created successfully")
        except Exception as e:
            print(f"Failed to create database {db_name}: {e}")


    def connect_to_database(self, db_name): 
        """Connect to a specific database.""" 
        self.select_database(db_name)

        

    def sql_dtype_map(self):
       """Constant SQL Data Type Mapping Details"""
       dtype_map_dict = {
                        'float64':'DECIMAL(20, 2)', 
                        'int64':'INTEGER(225)', 
                        'object' : "VARCHAR(225)"  
                        }
       
       return dtype_map_dict
    


    
    def create_tbl_script (self, df, table_name, primary_key=None):
        "Generate Create table Sql Script for any dataframes"
        columns = []
        sql = f"""drop table if exists {table_name}; \nCREATE TABLE {table_name} (\n"""
        for col in df.columns:
            col = col.replace(" ","_") ## Check Back 

            sql_dtype = self.sql_dtype_map()[str(df[col].dtype)]
            if col == primary_key:
                column_syntax = f"{col} {sql_dtype} PRIMARY KEY AUTO_INCREMENT\n"
            else:
                column_syntax = f"{col} {sql_dtype}\n"
            columns.append(column_syntax)
        columns_syntax = ",".join(columns) # Adding new line and indentation for better readability 
        sql +=  columns_syntax + ");" 
        print(sql)
        return sql


 
    
    def create_table(self, create_table_query, table_name):
        """Create a table in the database"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(create_table_query)
            print(f"Table [{table_name}] created successfully.")
            # cursor.close()
        except Exception as e:
            print(f"Error creating table: {e}")




    def insert_record_from_df (self, table_name, df):
        """Inserts the records from Dataframe to the Table """
        try:  
            if self.conn is None or not self.conn.is_connected():
                print("Reconnecting to MySQL...")
                self.connect()
            cursor = self.conn.cursor()
            for i, row in df.iterrows():
                # Create an INSERT query for each row
                row_values = row.astype(str).values.tolist()
                
                placeholders = ", ".join(["%s"] * len(row))
                columns = ", ".join(row.index)
                insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                cursor.execute(insert_query, tuple(row_values))
            self.conn.commit()
            print(f"Inserted {len(df)} rows into {table_name} successfully.")
            # cursor.close()
        except Exception as e:
            print(f"Error inserting data: {e}")
            self.conn.rollback()
            # cursor.close()




    def select_table_query(self,query):
        """Get records from Mysql into the Dataframe"""
        try:
            if self.conn is None or not self.conn.is_connected():
                print("Reconnecting to MySQL...")
                self.connect()
                
            cursor = self.conn.cursor()
            cursor.execute(query)
            records = cursor.fetchall()
            # cursor.close()
            
            column_list =[desc[0] for desc in  cursor.description]
            df = pd.DataFrame(records, columns=column_list)
            return df
        except  mysql.connector.Error as e:
            print(f"Error reading data from MySQL: {e}")
            return None
        


    def connection_close(self):
        if self.conn:
            self.conn.close()
            print('MySql Connection Closed.........\n')
 
        


In [73]:


movie_df = pd.read_csv('movies.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';')
movie_df.dropna(inplace=True)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3795 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      3795 non-null   object 
 1   rating    3795 non-null   object 
 2   genre     3795 non-null   object 
 3   year      3795 non-null   int64  
 4   released  3795 non-null   object 
 5   score     3795 non-null   float64
 6   votes     3795 non-null   float64
 7   director  3795 non-null   object 
 8   writer    3795 non-null   object 
 9   star      3795 non-null   object 
 10  country   3795 non-null   object 
 11  budget    3795 non-null   int64  
 12  gross     3795 non-null   float64
 13  company   3795 non-null   object 
 14  runtime   3795 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 474.4+ KB


In [74]:
### Mysql Server Connection 

budget_schema_config = {
    "host":"127.0.0.1",
    "user":"analysis"

}

mysql_conn = MySqlConnection(**budget_schema_config)
mysql_conn.connect()

MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-01-09 11:05:26 PM


In [75]:
mysql_conn.create_datebase(db_name='Test')

Database Test created successfully


In [76]:
mysql_conn.select_database(db_name='Test')

Connected to database: Test


In [77]:
create_tbl_sql = mysql_conn.create_tbl_script(movie_df, table_name="movie", primary_key=None)

drop table if exists movie; 
CREATE TABLE movie (
name VARCHAR(225)
,rating VARCHAR(225)
,genre VARCHAR(225)
,year INTEGER(225)
,released VARCHAR(225)
,score DECIMAL(20, 2)
,votes DECIMAL(20, 2)
,director VARCHAR(225)
,writer VARCHAR(225)
,star VARCHAR(225)
,country VARCHAR(225)
,budget INTEGER(225)
,gross DECIMAL(20, 2)
,company VARCHAR(225)
,runtime DECIMAL(20, 2)
);


In [78]:

mysql_conn.create_table(create_table_query=create_tbl_sql,table_name='movie')

Table [movie] created successfully.


In [79]:
mysql_conn.select_database(db_name='Test')

Reconnecting to MySQL...
MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-01-09 11:05:26 PM
Connected to database: Test


In [80]:
mysql_conn.insert_record_from_df (table_name='movie', df =movie_df)

Inserted 3795 rows into movie successfully.


In [97]:
from pathlib import Path

data_set_path = 'D:\ML_NOTE_DATASET\Analysis_dataset'
data_set_path = Path(data_set_path)
data_set_path
zomato_df = pd.read_csv(f"{data_set_path}/zomato.csv", encoding='ISO-8859-1', on_bad_lines='skip')
zomato_df.rename(columns={ x : x.replace(" ","_") for  x in zomato_df.columns} ,inplace=True)
zomato_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant_ID         9551 non-null   int64  
 1   Restaurant_Name       9551 non-null   object 
 2   Country_Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality_Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average_Cost_for_two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has_Table_booking     9551 non-null   object 
 13  Has_Online_delivery   9551 non-null   object 
 14  Is_delivering_now     9551 non-null   object 
 15  Switch_to_order_menu 

In [103]:
zomato_df

,Restaurant_ID,Restaurant_Name,Country_Code,City,Address,Locality,Locality_Verbose,Longitude,Latitude,Cuisines,...,Currency,Has_Table_booking,Has_Online_delivery,Is_delivering_now,Switch_to_order_menu,Price_range,Aggregate_rating,Rating_color,Rating_text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,NamlÛ± Gurme,208,ÛÁstanbul,"Kemankeô Karamustafa Paôa Mahallesi, RÛ±htÛ±...",Karakí_y,"Karakí_y, ÛÁstanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz AÛôacÛ±,208,ÛÁstanbul,"Koôuyolu Mahallesi, Muhittin íìstí_ndaÛô Cadd...",Koôuyolu,"Koôuyolu, ÛÁstanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,Aôôk Kahve,208,ÛÁstanbul,"Kuruí_eôme Mahallesi, Muallim Naci Caddesi, N...",Kuruí_eôme,"Kuruí_eôme, ÛÁstanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [98]:
mysql_conn.create_datebase(db_name='Zomato')

Database Zomato created successfully


In [99]:
mysql_conn.connect_to_database(db_name='Zomato')

Connected to database: Zomato


In [101]:
table_name='Zomato_table'
primary_key='Restaurant_ID'
zomato_create_tbl_script = mysql_conn.create_tbl_script(df=zomato_df,table_name=table_name, primary_key=primary_key)

drop table if exists Zomato_table; 
CREATE TABLE Zomato_table (
Restaurant_ID INTEGER(225) PRIMARY KEY AUTO_INCREMENT
,Restaurant_Name VARCHAR(225)
,Country_Code INTEGER(225)
,City VARCHAR(225)
,Address VARCHAR(225)
,Locality VARCHAR(225)
,Locality_Verbose VARCHAR(225)
,Longitude DECIMAL(20, 2)
,Latitude DECIMAL(20, 2)
,Cuisines VARCHAR(225)
,Average_Cost_for_two INTEGER(225)
,Currency VARCHAR(225)
,Has_Table_booking VARCHAR(225)
,Has_Online_delivery VARCHAR(225)
,Is_delivering_now VARCHAR(225)
,Switch_to_order_menu VARCHAR(225)
,Price_range INTEGER(225)
,Aggregate_rating DECIMAL(20, 2)
,Rating_color VARCHAR(225)
,Rating_text VARCHAR(225)
,Votes INTEGER(225)
);


In [102]:
mysql_conn.create_table(zomato_df, table_name=table_name)

Error creating table: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


In [ ]:
employee_df  = mysql_conn.select_table_query("Select * from employees")
employee_df

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.00,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.00,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.00,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9000.00,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.00,103.0,6
5,105,David,Austin,david.austin@sqltutorial.org,590.423.4569,1997-06-25,9,4800.00,103.0,6
6,106,Valli,Pataballa,valli.pataballa@sqltutorial.org,590.423.4560,1998-02-05,9,4800.00,103.0,6
7,107,Diana,Lorentz,diana.lorentz@sqltutorial.org,590.423.5567,1999-02-07,9,4200.00,103.0,6
8,108,Nancy,Greenberg,nancy.greenberg@sqltutorial.org,515.124.4569,1994-08-17,7,12000.00,101.0,10
9,109,Daniel,Faviet,daniel.faviet@sqltutorial.org,515.124.4169,1994-08-16,6,9000.00,108.0,10


In [31]:

mysql_conn.connection_close()

MySql Connection Closed.........

